# Inspect and Verify that the Source Table is reasonable
Michael Wood-Vasey

To test the Source Table before it's in production we use a copy of our local branch of `gcr-catalogs`, `issues/274`.
Replace the path below in the `sys.path.insert` with the location of your own local checkout.

In [ ]:
import sys
sys.path.insert(0, '/global/homes/w/wmwv/local/lsst/gcr-catalogs')

In [ ]:
import os

import numpy as np
import pandas as pd

import GCRCatalogs

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

In [ ]:
object_reader = 'dc2_object_run1.2i'
source_reader = 'dc2_source_run1.2i'
# source_reader_config = {'base_dir': '/global/cscratch1/sd/wmwv/DC2/Run1.2i/src_visit'}
source_reader_config = {}

In [ ]:
dpdd_parquet_file = 'dpdd_dc2_object_run1.2i.parquet'
dpdd_parquet_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.2i/object_catalog_new'
object_table_parquet_file = os.path.join(dpdd_parquet_dir, dpdd_parquet_file)

source_table_parquet_file = '../test_src_visit_241985.parquet'

In [ ]:
# object_table = pd.read_parquet(object_table_parquet_file)
# source_table = pd.read_parquet(source_table_parquet_file)

In [ ]:
object_table = GCRCatalogs.load_catalog(object_reader)

In [ ]:
source_table = GCRCatalogs.load_catalog(source_reader, config_overwrite=source_reader_config)

In [ ]:
# source_table.generate_schema_yaml()

In [ ]:
print(len(object_table), len(source_table))

In [ ]:
# Match the object ID and check that we really got the right thing in the source table ID

In [ ]:
bright_ids = pd.DataFrame(source_table.get_quantities(['sourceId', 'objectId'], native_filters=['mag_err < 0.05']))

In [ ]:
all_ids = pd.DataFrame(source_table.get_quantities(['sourceId', 'objectId', 'ra', 'dec', 'mag', 'mag_err']))

In [ ]:
all_ids['mag_err']

In [ ]:
bright_ids = all_ids[all_ids['mag_err'] < 0.05]

In [ ]:
len(bright_ids)

In [ ]:
plt.hexbin(bright_ids['mag'], bright_ids['mag_err'])
plt.xlabel('mag')
plt.ylabel('mag_err')
plt.xlim(14, 25)

### Looking for objectId for one source
Let's look for all matching Sources for one object.
Let's just take the first entry from the bright_ids, assuming bright sources should be detected in the coadd and have an objectId.

Basic checks:
  * Do the RA, Dec values agree across the Source measurements?
  * Do they agree with the Object Table RA, Dec?

In [ ]:
this_object_id = bright_ids['objectId'].values[0]
these_sources = bright_ids[bright_ids['objectId'] == this_object_id] 

In [ ]:
print(these_sources[['sourceId', 'objectId', 'ra', 'dec']])

In [ ]:
object_df = pd.DataFrame(object_table.get_quantities(['id', 'ra', 'dec']))
this_object = object_df[object_df['id'] == this_object_id]

In [ ]:
this_object

Yes!  Looks good.

In [ ]:
source_table.list_all_quantities()

In [ ]:
resid = source_table['mag'] - (-2.5*np.log10(source_table['slot_PsfFlux_instFlux']/source_table['fluxmag0']))

In [ ]:
print(resid)

In [ ]:
source_table['slot_PsfFlux_instFlux']

In [ ]:
source_table['slot_ApFlux_instFlux']